In [ ]:
import os
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

URL = "https://www.magnespress.co.il/book/<id>/read"  # Open the book in the online reader and copy url
OUTPUT_DIR = "pdf_outputs" # Dir for scraped images 
PDF_NAME = "merged.pdf" # Filename for resulting pdf
LOGIN = "<email>" # magnespress.co.il login (email)
PASSD = "<password>" # magnespress.co.il password

In [ ]:
# Initialize driver
# Note that this needs geckodriver in the directory

os.makedirs(OUTPUT_DIR, exist_ok=True)
firefox_options = webdriver.FirefoxOptions()
firefox_options.add_argument("--headless")
driver = webdriver.Firefox(options=firefox_options)

In [ ]:
# Open book

driver.get(URL)
time.sleep(5)

In [ ]:
# Login to account

login_element = driver.find_element(By.XPATH, "/html/body/div[7]/div/div[2]/div/div/div/div[1]/div[1]/input")
passd_element = driver.find_element(By.XPATH, "/html/body/div[7]/div/div[2]/div/div/div/div[1]/div[2]/input")

login_element.send_keys(LOGIN)
passd_element.send_keys(PASSD)

login_button = driver.find_element(By.XPATH, "/html/body/div[7]/div/div[2]/div/div/div/div[1]/div[4]/button[1]")
login_button.click()
time.sleep(5)

In [ ]:
# Find number of pages and scroll to the start

viewer_element = driver.find_element(By.ID, "viewerContainer")
viewer_element.click()
time.sleep(1)

pages_element = driver.find_element(By.XPATH, "/html/body/div[3]/div[2]/div/span")
num_pages = int(pages_element.text.strip())

pages_input = driver.find_element(By.XPATH, "/html/body/div[3]/div[2]/input")
pages_input.send_keys("1")
pages_input.send_keys(Keys.ENTER)

viewer_element.click()
time.sleep(1)

In [ ]:
# Scroll through pages and take screenshot

for page in range(1, num_pages+1):
    print(f"Processing {page} out of {num_pages}")
    
    page_element = driver.find_element(By.ID, f'pageContainer{page}')
    
    page_size = page_element.size
    page_width = page_size['width']
    page_height = page_size['height']
    
    driver.set_window_size(page_width*2, page_height*2)
    driver.execute_script("arguments[0].scrollIntoView();", page_element)
    
    page_element.screenshot(f"pdf_outputs/page_{page}.png")
    
    if page != num_pages:
        next_element = driver.find_element(By.ID, "pageLeft")
        next_element.click()
        time.sleep(1)

In [ ]:
driver.quit()

In [ ]:
import img2pdf

images = [os.path.join(OUTPUT_DIR, f) for f in os.listdir(OUTPUT_DIR) if f.endswith(('.png'))]

with open(PDF_NAME, "wb") as f:
    f.write(img2pdf.convert(images))
    
# Uncomment to remove scraped images at the end
#os.rmdir(OUTPUT_DIR)